# Análista de Empleo con IA

### Importación de librerias

In [97]:
import json
from bs4 import BeautifulSoup
import re
from openai import OpenAI
from api_key import API_KEY

### Carga de datos

In [8]:
file = "../docs/cv_toy.txt"

In [9]:
with open(file) as f:
    soup = BeautifulSoup(f.read())

In [10]:
soup

447 Sage Hall						E-mail mo19@ cornell.edu
Johnson Graduate School of Management		(607) 255-3645 Phone
Cornell University					(607) 254-4590 FAX
Ithaca, New York 14853 		web page: www.johnson.cornell.edu/	facultybios/ohara.html                  
Citizenship:  U.S., Ireland	https://ssrn.com/author=16137
https://orcid.org/0000-0003-2563-7748

Education

Ph.D. (Finance), Kellogg Graduate School of Management, 
   Northwestern University, 1979

M.A. (Economics), Northwestern University, 1976

B.S. (Economics), University of Illinois, 1975
	       University of Manchester

Honorary Degrees

Doctor of Laws Honoris Causa, awarded by University College Dublin, 2015.

Doctorate Honoris Causa, awarded by the Universität Bern, Switzerland, 2012.

Doctorate Honoris Causa, awarded by the Facultés Universitaires Catholiques à Mons (FUCAM), Belgium, 2007.

Positions Held

Robert W. Purcell Professor of Finance and Professor of Economics,
Cornell University 
Professor (1989- current), Associate Profe

### Experimento 1 - Obtener un resumen del CV.

Con este experimento se busca que una IA generativa tome un texto y lo resuma, simplemente se quiere probar que la api funciona y devuelve una respuesta con lo que se pregunta de manera sencilla y coherente a lo que se le pide.


In [11]:
client = OpenAI(api_key=API_KEY, base_url="https://api.perplexity.ai")


In [16]:
messages = [
    {"role": "system", "content": "You are a helpful assistant that summarizes text."},
    {"role": "user", "content": f"Please provide a concise summary of the following text:\n\n{soup}"}
]

response = client.chat.completions.create(
    model="sonar-pro",
    messages=messages,
    max_tokens=500  
)


**Respuesta con max_tokens = 200**

In [13]:
response.choices[0].message.content

'Here is a concise summary of the key information from Maureen O\'Hara\'s curriculum vitae:\n\nMaureen O\'Hara is the Robert W. Purcell Professor of Finance at Cornell University\'s Johnson Graduate School of Management. She holds a Ph.D. in Finance from Northwestern University and has received honorary doctorates from three European universities. \n\nHer research focuses on market microstructure, and she has authored numerous influential publications in this field, including the book "Market Microstructure Theory". She has served as president of several major finance associations, including the American Finance Association.\n\nO\'Hara has received many awards and honors for her research and teaching, including multiple Smith'

**Respuesta con max_tokens = 500**

In [17]:
summary = response.choices[0].message.content
print(summary)

Maureen O'Hara is a prominent financial economist and professor at Cornell University. Key points about her background and career include:

1. Current position: Robert W. Purcell Professor of Finance at Cornell University's Johnson Graduate School of Management.

2. Education: Ph.D. in Finance from Northwestern University, M.A. in Economics from Northwestern, and B.S. in Economics from the University of Illinois.

3. Research focus: Market microstructure, with recent work on ETFs, liquidity in corporate bond markets, and corporate governance in banks.

4. Authored numerous journal articles and books, including the influential "Market Microstructure Theory."

5. Held leadership positions in major finance associations, including President of the American Finance Association.

6. Received multiple awards and honors for her research and teaching.

7. Served on corporate and advisory boards, including TIAA and the National Bureau of Economic Research.

8. Extensive publication record coveri

### Experimento 2 - Extraer datos del CV en formato JSON

En este caso el objetivo es demostrar que puedo estructurar los datos de tal manera qué, una vez estos han sido procesados, la api devuelva la misma información en formato JSON.  Esto permitirá alimentar una base de datos con esta información de forma automática.  Al hacerlo de esta forma se pueden también extraer palabras claves relacionadas con las actividades que realiza, trabajo actual, premios etc, demostrando un uso importante de una IA con una aplicación en el mundo real para un negocio real.


In [90]:
def process_cv(cv_text):

    prompt = f"""
    Extract the following information from the given CV and format it as JSON:
    - name stored as "name"
    - summary (about 200 words), stored as "summary"
    - experience (list of jobs with company, position, and duration) stored as "experience"
    - city_of_residence, stored as "city"
    - country_of_residence stored as country
    - current_job, with variable name "current_job"
    - employers (list), with variable name "employers"
    - keywords (list based on job experience), stored as "keywords"
    - education (list of studies), stored as "education"
    - awards (list), stored as "awards"
    

    CV text: {cv_text}

    Provide the output in valid JSON format without any additional text.
    """

    messages = [
        {"role": "system", "content": "You are an AI assistant that extracts and structures CV information."},
        {"role": "user", "content": prompt}
    ]

    response = client.chat.completions.create(
        model="sonar-pro",
        messages=messages,
        max_tokens=2000
    )
    
    return response

In [91]:
result = process_cv(soup)

In [92]:
result_content = response.choices[0].message.content

In [93]:
result_content

'Maureen O\'Hara is a prominent financial economist and professor at Cornell University. Key points about her background and career include:\n\n1. Current position: Robert W. Purcell Professor of Finance at Cornell University\'s Johnson Graduate School of Management.\n\n2. Education: Ph.D. in Finance from Northwestern University, M.A. in Economics from Northwestern, and B.S. in Economics from the University of Illinois.\n\n3. Research focus: Market microstructure, with recent work on ETFs, liquidity in corporate bond markets, and corporate governance in banks.\n\n4. Authored numerous journal articles and books, including the influential "Market Microstructure Theory."\n\n5. Held leadership positions in major finance associations, including President of the American Finance Association.\n\n6. Received multiple awards and honors for her research and teaching.\n\n7. Served on corporate and advisory boards, including TIAA and the National Bureau of Economic Research.\n\n8. Extensive public

In [94]:
content = result.choices[0].message.content
content

'{\n  "name": "Maureen O\'Hara",\n  "summary": "Maureen O\'Hara is a distinguished finance professor and researcher with extensive experience in market microstructure, financial regulation, and corporate governance. She has held prominent positions at Cornell University and other institutions, authored influential books and research papers, and received numerous awards for her contributions to financial economics. O\'Hara has served as president of major finance associations and held board positions at financial firms and organizations. Her expertise spans areas such as high-frequency trading, cryptocurrency markets, and corporate bond trading.",\n  "experience": [\n    {\n      "company": "Cornell University",\n      "position": "Robert W. Purcell Professor of Finance and Professor of Economics",\n      "duration": "1979 - current"\n    },\n    {\n      "company": "University of Technology Sydney",\n      "position": "Professor of Finance",\n      "duration": "2016 - 2021"\n    },\n  

In [98]:
def clean_and_parse_json(json_string):
    # Remover comas finales
    json_string = re.sub(r',\s*}', '}', json_string)
    json_string = re.sub(r',\s*]', ']', json_string)
    
    # Asegurar que las cadenas de texto cierren con corchetes
    if not json_string.strip().endswith('}'):
        json_string = json_string.strip() + '}'
    
    # Formatear como JSON
    try:
        return json.loads(json_string)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        print("Problematic part of the string:")
        print(json_string[max(0, e.pos-50):min(len(json_string), e.pos+50)])
        return None


json_object = clean_and_parse_json(content)

if json_object:
    cv = json.dumps(json_object, indent=2)
    print("Successfully parsed JSON:")
    print(cv)
else:
    print("Failed to parse JSON")


Successfully parsed JSON:
{
  "name": "Maureen O'Hara",
  "summary": "Maureen O'Hara is a distinguished finance professor and researcher with extensive experience in market microstructure, financial regulation, and corporate governance. She has held prominent positions at Cornell University and other institutions, authored influential books and research papers, and received numerous awards for her contributions to financial economics. O'Hara has served as president of major finance associations and held board positions at financial firms and organizations. Her expertise spans areas such as high-frequency trading, cryptocurrency markets, and corporate bond trading.",
  "experience": [
    {
      "company": "Cornell University",
      "position": "Robert W. Purcell Professor of Finance and Professor of Economics",
      "duration": "1979 - current"
    },
    {
      "company": "University of Technology Sydney",
      "position": "Professor of Finance",
      "duration": "2016 - 2021"
 

### Experimento 3 - Evaluar si un candidato es apto para un cargo basado en el contenido de un CV con un perfil inadecuado para el cargo.

Ahora, en éste experimento, se quiere es modificar el prompt de tal manera que el asistente de IA lea un texto de una vacante, y basado en esa información evalúe si el aplicante es adecuado o no para ese trabajo.

In [99]:
def create_job_evaluation_prompt(job_title, job_description, applicant_data, language="English"):
    prompt = f"""
    Analyze the following JSON data of a job applicant for the position of "{job_title}":

    {applicant_data}

    The job title "{job_title}" is in {language}. Here's a brief description of the role:
    {job_description}

    Based on this information, evaluate the candidate's suitability for the role by considering the following factors:
    1. Relevant work experience
    2. Required skills for the position
    3. Language proficiency (if applicable)
    4. Education and qualifications
    5. Any additional skills or experiences that could be beneficial for this role

    Provide a concise assessment of the candidate's strengths and weaknesses for this position, and conclude with a recommendation on whether they are suitable for the role. Rate their overall suitability on a scale of 1-10, with 10 being highly suitable.

    If the job title or description is in a language other than English, please consider any cultural or linguistic factors that may be relevant to the evaluation.
    """
    return prompt

In [100]:
job_title = "Auxiliar de ventas en panadería"
job_description = """
Responsabilidades:
- Atender a los clientes y procesar ventas
- Mantener el área de ventas limpia y organizada
- Ayudar en la preparación y presentación de productos
- Manejar transacciones en efectivo y con tarjeta
- Trabajar en horarios flexibles, incluyendo mañanas tempranas y fines de semana

Requisitos:
- Experiencia previa en ventas o atención al cliente (preferible)
- Habilidades matemáticas básicas
- Excelentes habilidades de comunicación en español
- Capacidad para trabajar en equipo
- Actitud positiva y orientada al servicio
"""

applicant_data = cv

evaluation_prompt = create_job_evaluation_prompt(job_title, job_description, applicant_data, language="Spanish")

In [102]:
# Use the prompt with Perplexity API
response = client.chat.completions.create(
    model="sonar-pro",
    messages=[
        {"role": "system", "content": "You are an AI assistant that evaluates job candidates."},
        {"role": "user", "content": evaluation_prompt}
    ],
    max_tokens=500
)

In [103]:
print(response.choices[0].message.content)

Based on the provided information about Maureen O'Hara and the requirements for the "Auxiliar de ventas en panadería" position, I can provide the following assessment:

1. Relevant work experience: 
The candidate has no relevant experience in retail sales or bakery work. Her extensive background is in finance and academia.

2. Required skills:
- Customer service: No evident experience
- Basic math skills: Likely strong given her finance background
- Teamwork: Possibly developed through academic collaborations
- Positive attitude: Unable to assess from given information

3. Language proficiency:
No information provided about Spanish language skills, which are crucial for this role.

4. Education and qualifications:
The candidate is highly overqualified with a Ph.D. in Finance and multiple professorships.

5. Additional beneficial skills:
Her background in finance might be useful for handling transactions and managing inventory, but is largely irrelevant for this position.

Strengths:
- 

### Experimento 4 - Evaluar lo anterior, pero está vez con un perfil adecuado para el cargo y cambio del lenguaje del prompt.


In [104]:
def create_job_evaluation_prompt_lang(job_title, job_description, applicant_data, language="English", response_language="English"):
    prompt = f"""
    Analiza los siguientes datos JSON de un candidato para el puesto de "{job_title}":

    {applicant_data}

    El título del trabajo "{job_title}" está en {language}. Aquí hay una breve descripción del puesto:
    {job_description}

    Basándote en esta información, evalúa la idoneidad del candidato para el puesto considerando los siguientes factores:
    1. Experiencia laboral relevante
    2. Habilidades requeridas para el puesto
    3. Dominio del idioma (si aplica)
    4. Educación y cualificaciones
    5. Cualquier habilidad o experiencia adicional que pueda ser beneficiosa para este puesto

    Proporciona una evaluación concisa de las fortalezas y debilidades del candidato para este puesto, y concluye con una recomendación sobre si es adecuado para el puesto. Califica su idoneidad general en una escala del 1 al 10, siendo 10 altamente adecuado.

    Si el título del trabajo o la descripción están en un idioma diferente al español, por favor considera cualquier factor cultural o lingüístico que pueda ser relevante para la evaluación.

    Por favor, proporciona tu respuesta en {response_language}.
    """
    return prompt

In [105]:
# Example usage
job_title = "Auxiliar de ventas en panadería"
job_description = """
Responsabilidades:
- Atender a los clientes y procesar ventas
- Mantener el área de ventas limpia y organizada
- Ayudar en la preparación y presentación de productos
- Manejar transacciones en efectivo y con tarjeta
- Trabajar en horarios flexibles, incluyendo mañanas tempranas y fines de semana

Requisitos:
- Experiencia previa en ventas o atención al cliente (preferible)
- Habilidades matemáticas básicas
- Excelentes habilidades de comunicación en español
- Capacidad para trabajar en equipo
- Actitud positiva y orientada al servicio
"""

applicant_data = """
{
    "name": "Maria González",
    "experience": [
        {
            "position": "Cajera",
            "company": "Supermercado Local",
            "duration": "2 años"
        },
        {
            "position": "Asistente de Cafetería",
            "company": "Café del Barrio",
            "duration": "1 año"
        }
    ],
    "education": "Bachillerato completo",
    "skills": ["Atención al cliente", "Manejo de caja", "Trabajo en equipo"],
    "languages": ["Español (nativo)"],
    "availability": "Flexible, incluyendo fines de semana"
}
"""

In [106]:
evaluation_prompt = create_job_evaluation_prompt_lang(job_title, 
                                                      job_description, 
                                                      applicant_data, 
                                                      language="Spanish", 
                                                      response_language="Español")

In [110]:
# Use the prompt with Perplexity API
response = client.chat.completions.create(
    model="sonar-pro",
    messages=[
        {"role": "system", "content": "Eres un asistente de IA que evalúa candidatos para puestos de trabajo. Por favor, responde en español."},
        {"role": "user", "content": evaluation_prompt}
    ],
    max_tokens=600
)

In [111]:
print(response.choices[0].message.content)

Evaluación del candidato Maria González para el puesto de Auxiliar de ventas en panadería:

1. Experiencia laboral relevante:
Maria tiene experiencia relevante como cajera en un supermercado y asistente de cafetería, lo cual es beneficioso para el puesto de auxiliar de ventas en panadería. Su experiencia en atención al cliente y manejo de caja es directamente aplicable[1][2].

2. Habilidades requeridas:
Las habilidades de Maria en atención al cliente, manejo de caja y trabajo en equipo se alinean bien con los requisitos del puesto[2].

3. Dominio del idioma:
Maria es hablante nativa de español, lo cual cumple con el requisito de excelentes habilidades de comunicación en español[2].

4. Educación y cualificaciones:
Maria ha completado el bachillerato, lo cual es suficiente para este puesto[2].

5. Habilidades adicionales beneficiosas:
Su experiencia en cafetería podría ser útil para ayudar en la preparación y presentación de productos de panadería[1].

Fortalezas:
- Experiencia relevant

### Experimento 5 - Similar al anterior, pero considerando la localización de la oferta de trabajo y la localización del postulante.


In [130]:
def create_job_evaluation_prompt_lang_loc(job_title, job_description, applicant_data, language="English", response_language="English"):
    prompt = f"""
    Analiza los siguientes datos JSON de un candidato para el puesto de "{job_title}":

    {applicant_data}

    El título del trabajo "{job_title}" está en {language}. Aquí hay una breve descripción del puesto:
    {job_description}

    Basándote en esta información, evalúa la idoneidad del candidato para el puesto considerando los siguientes factores:
    1. Experiencia laboral relevante
    2. Habilidades requeridas para el puesto
    3. Dominio del idioma (si aplica)
    4. Educación y cualificaciones
    5. Cualquier habilidad o experiencia adicional que pueda ser beneficiosa para este puesto
    6. Ubicación del candidato en relación con el lugar de trabajo

    Proporciona una evaluación concisa de las fortalezas y debilidades del candidato para este puesto, y concluye con una recomendación sobre si es adecuado para el puesto. Califica su idoneidad general en una escala del 1 al 10, siendo 10 altamente adecuado.

    Si el título del trabajo o la descripción están en un idioma diferente al español, por favor considera cualquier factor cultural o lingüístico que pueda ser relevante para la evaluación.

    Por favor, proporciona tu respuesta en {response_language}.
    """
    return prompt

In [131]:
# Caso de uso
job_title = "Barman en Club en Bogotá"
job_description = """
Responsabilidades:
- Preparar y servir bebidas alcohólicas y no alcohólicas en un ambiente de club nocturno
- Mantener el área del bar limpia y organizada
- Manejar transacciones en efectivo y con tarjeta
- Verificar la identificación de los clientes para asegurar el cumplimiento de la edad legal para beber
- Trabajar en horarios nocturnos y fines de semana
- Mantener un inventario preciso de las existencias del bar

Requisitos:
- Experiencia previa como barman o en servicios de bar (preferible)
- Conocimiento de cócteles clásicos y modernos
- Excelentes habilidades de comunicación en español
- Capacidad para trabajar en un ambiente de ritmo rápido
- Actitud positiva y orientada al servicio al cliente
- Disponibilidad para trabajar en horarios nocturnos y fines de semana en Bogotá
"""

applicant_data = """
{
    "name": "Pedro Pérez",
    "residence": "Cartagena, Colombia",
    "experience": [
        {
            "position": "Barman",
            "company": "La Cervecería",
            "location": "Cartagena, Colombia",
            "duration": "2 años"
        },
        {
            "position": "Asistente de Catering",
            "company": "Eventos Elegantes",
            "location": "Medellín, Colombia",
            "duration": "1 año"
        },
        {
            "position": "Mesero",
            "company": "Restaurante El Sabor",
            "location": "Bogotá, Colombia",
            "duration": "1 año y 6 meses"
        }
    ],
    "education": "Curso de Bartender Profesional, Escuela de Coctelería de Bogotá",
    "skills": ["Preparación de cócteles", "Atención al cliente", "Manejo de caja", "Trabajo en equipo"],
    "languages": ["Español (nativo)", "Inglés (básico)"],
    "availability": "Disponible para horarios nocturnos y fines de semana"
}
"""

In [132]:
evaluation_prompt = create_job_evaluation_prompt_lang_loc(job_title, job_description, applicant_data, language="Spanish", response_language="Español")

In [133]:
# Use the prompt with Perplexity API
response = client.chat.completions.create(
    model="sonar-pro",
    messages=[
        {"role": "system", "content": "Eres un asistente de IA que evalúa candidatos para puestos de trabajo. Por favor, responde en español."},
        {"role": "user", "content": evaluation_prompt}
    ],
    max_tokens=600
)

In [134]:
print(response.choices[0].message.content)

Evaluación del candidato Pedro Pérez para el puesto de Barman en Club en Bogotá:

1. Experiencia laboral relevante:
Pedro tiene 2 años de experiencia como barman, lo cual es directamente relevante para el puesto. También cuenta con experiencia en catering y como mesero, que aportan habilidades transferibles al trabajo de barman.

2. Habilidades requeridas:
El candidato posee habilidades clave como preparación de cócteles, atención al cliente, manejo de caja y trabajo en equipo, que son esenciales para el puesto.

3. Dominio del idioma:
Pedro es nativo en español, lo cual cumple con el requisito de excelentes habilidades de comunicación en español.

4. Educación y cualificaciones:
Ha completado un Curso de Bartender Profesional, lo que demuestra formación específica en el área.

5. Habilidades adicionales beneficiosas:
Su experiencia en catering podría ser útil para eventos especiales en el club.

6. Ubicación:
Pedro reside actualmente en Cartagena, lo que podría ser un inconveniente pa